# Classification Model
This model was not used for the final product. Here we train a classification model based on resnet34. The model ends up being able to predict whether it needs to steer left, steer right, brake or go straight ahead. This code no longer works because the required folder structure no longer exists.

In [ ]:
import os
import shutil
from fastai.vision.all import *

from model import ScreenCapture
from classification_model import GamepadSimulator

We start by dividing the data into a train set and a validation set. Originally we also had the data divided into four folders, each representing a label: gas, steer_left, steer_right or break. This was done in the screen_grab.py file, but is no longer there as it is was not required for our final model.

In [ ]:
def create_validation_set(train_dir, val_dir, split_ratio=0.2):
    # Create validation directory if it doesn't exist
    os.makedirs(val_dir, exist_ok=True)

    # Iterate over each label folder in the train directory
    for label in os.listdir(train_dir):
        label_path = os.path.join(train_dir, label)

        # Check if it is a directory (label folder)
        if os.path.isdir(label_path):
            images = os.listdir(label_path)

            # Calculate the number of validation images (20% of total)
            val_size = int(len(images) * split_ratio)

            # Select the validation images
            val_images = images[:val_size]

            # Create label subdirectory in the validation directory
            os.makedirs(os.path.join(val_dir, label), exist_ok=True)

            # Move validation images to validation directory
            for image in val_images:
                shutil.move(os.path.join(label_path, image), os.path.join(val_dir, label, image))

if __name__ == "__main__":
    # Define the train directory and the validation directory
    train_directory = './screen_caps/train'  
    val_directory = './screen_caps/valid'      

    # Create the validation set
    create_validation_set(train_directory, val_directory, split_ratio=0.2)


We created a simple model based on resnet34. We first resize the images for uniformity and apply some simple transformations. We then train the model for 10 epochs with a batch size of 64,

In [ ]:
from fastai.vision.all import *

path = Path("./screen_caps")

# Create a DataLoader for training
dls = ImageDataLoaders.from_folder(path, train='train', valid='valid', 
                                   item_tfms=Resize(224), batch_tfms=aug_transforms(), 
                                   bs=64)

# Define the model using FastAI's `cnn_learner` function
learner = cnn_learner(dls, resnet34, metrics=accuracy)


# Train the model
learner.fine_tune(10) 

learner.save('model')

Next up we use this model to make predictions that can be used to play the game.

In [ ]:
def load_model():
    """Load the trained model"""
    learner = cnn_learner(dls, resnet34, metrics=accuracy)
    learner.load('model') 
    return learner

def drive_game(learner, screen_capture, gamepad_simulator, duration=60):
    """Use the model to predict actions and simulate gamepad inputs"""
    start_time = time.time()
    
    while time.time() - start_time < duration:
        # Capture the game screen
        frame = screen_capture.capture_frame()
        
        # Convert the frame to a PIL image
        pil_img = PILImage.create(frame)
        if pil_img.mode == 'RGBA':
            pil_img = pil_img.convert('RGB')
        
        # Predict the next action using the trained model
        pred, _, _ = learner.predict(pil_img)
        
        # Extract the predicted action (you can modify this part to map to your controller actions)
        action = pred
        print(f"Predicted Action: {action}")
        
        # Simulate the controller inputs based on the prediction (implement your logic)
        if action == 'steer_left':
            gamepad_simulator.set_input(lx=-1, lt=0, rt=0)
        elif action == 'steer_right':
            gamepad_simulator.set_input(lx=1, lt=0, rt=0)
        elif action == 'brake':
            gamepad_simulator.set_input(lx=0, lt=1, rt=0)
        elif action == 'gas':
            gamepad_simulator.set_input(lx=0, lt=0, rt=1)
        
        # Sleep for a short period to control the simulation speed
        time.sleep(0.1)

if __name__ == '__main__':
    # Initialize components
    screen_capture = ScreenCapture()
    gamepad_simulator = GamepadSimulator()
    learner = load_model()

    # Start driving the game for 60 seconds (you can adjust the duration)
    drive_game(learner, screen_capture, gamepad_simulator, duration=60)


This model is now able to choose one of four actions: gas, steer_left, steer_right or break. We however decided to go with joystick inputs, which requires a regression model. Therefore, this model got abandoned.